In [ ]:
# load npy
# turn to npy to do all of these in parallel, one section at a time, white space non tissue
# ink removal
# label expansion with bayes
# downsize original and labeled image? no! maybe by 2

# notes
# ink removal here, in future at beginning
# next script, stitch and open sea dragon; v2 go 3D model; start with patient and do ordering; rotation matrix from shapes
# stain norm at beginning of entire pipeline
# stain norm here? in future do at beginning



In [1]:
# given posterior distribution ; may also help out with histobayes
# randomly sample from predictive posterior during training
# add training 
# new graph construction given more datapoints in between
# single graph prediction propagation, bce with probability
# trim new points at beginning with tissue mask to avoid grabbing 256 over, which propagates to rest of slide, else maybe initialize with argmaxed label propagation first

# start with one graph


import copy, torch, os
from sklearn.decomposition import PCA
import copy, numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
import numpy.ma as ma
from sklearn.metrics import classification_report, f1_score
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelBinarizer
from itertools import product
from sklearn.neighbors import radius_neighbors_graph
from torch_geometric.utils import from_scipy_sparse_matrix
from torch_geometric.data import Data
import torch.nn as nn
from torch.nn import functional as F
from torch_geometric.nn import APPNP, AGNNConv, GATConv
import cv2

import numpy as np
import tqdm

import matplotlib,matplotlib.pyplot as plt


%matplotlib inline
matplotlib.rcParams['figure.dpi']=300


In [2]:



class Net(nn.Module):
    def __init__(self,n_layers=3,K=5,alpha=0.2,self_loops=False):
        super().__init__()
        assert n_layers>=2
        self.propagate=nn.ModuleList([APPNP(K=K,alpha=alpha,add_self_loops=self_loops) for i in range(n_layers)])#AGNNConv()
        
    def forward(self,x,edge_index,mask):
        x_mask=x[mask]
        for layer in self.propagate[:-1]:
            x=layer(x,edge_index)
            x_mask=x[mask]
            # relu?
        x_mask=x[mask]
        return self.propagate[-1](x,edge_index)
        




no_pretrain=False
j=4
cv_splits='bcc/graph_datasets/cv_splits_256_pretrain.pkl'
cv_splits=pickle.load(open(cv_splits,'rb'))

prediction_file="bcc/predictions/4.predictions.pth"
graph_dataset=pickle.load(open("bcc/graph_datasets/pretrain_graph_data_256.pkl",'rb'))
print(prediction_file)
val_idx=cv_splits[j]['train_idx']
test_idx=cv_splits[j]['val_idx']
graphs=torch.load(prediction_file)


val_graphs=[graphs[i] for i in val_idx]#[:len(val_idx)]
test_graphs=[graphs[i] for i in test_idx]#graphs#[len(val_idx):]
all_graphs=val_graphs+test_graphs



bcc/predictions/4.predictions.pth


In [3]:
# val_idx

In [4]:
# test_idx

In [5]:
# components=(graph_dataset['df']['ID']+"_"+graph_dataset['df']['component'].astype(str)).unique()

In [4]:
# import pandas as pd
# pd.Series(np.vectorize(lambda x: x.split('_')[0])(components)).value_counts()

In [5]:
# k="163"
# df_100=graph_dataset['df'].loc[np.vectorize(lambda x: x.split('_')[0]==k)(graph_dataset['df']['ID'])]

In [6]:
# "163_A1a"

In [7]:
# df_100.groupby(['ID','component']).count()

In [ ]:
# import numpy as np, cv2
# arr=cv2.resize(np.load("bcc/inputs/163_B2b.npy"),None,fx=1/6,fy=1/6)#cv2.cvtColor(,cv2.COLOR_BGR2RGB)

In [ ]:
# plt.imshow(arr)

In [ ]:
# idx
#np.hstack([val_idx,test_idx])

In [ ]:
# [slide.id for slide in graph_dataset['graph_dataset']]

In [ ]:
# val_graphs

In [ ]:


# 



# [(x,y) for x,y in ]

# val_idx

In [ ]:
# ! ps aux | grep f003k8w

In [ ]:
# graph_dataset['graph_dataset']

In [16]:

# graph_ids
# I maybe need to trace back each component ?? 

array([0, 1, 2, 0, 1, 2, 3, 4, 0, 1, 0, 1, 2, 3, 0, 1, 2, 3, 4, 0, 1, 2,
       0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 5, 0, 1, 0, 1, 2, 3, 4, 0, 1, 0, 1,
       2, 3, 4, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 0, 1, 2, 0, 1, 0, 1, 0,
       1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 0, 1, 0, 1, 2, 3, 4, 5,
       0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 4, 0, 1, 2, 0, 1,
       2, 0, 1, 2, 0, 1, 2, 3, 0, 1, 2, 3, 4, 0, 1, 2, 3, 0, 1, 2, 3, 0,
       1, 2, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 1, 2, 3, 4, 0, 1, 2, 0, 1, 2,
       0, 1, 2, 0, 1, 0, 1, 2, 3, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 0,
       1, 2, 0, 1, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3,
       4, 5, 0, 1, 2, 3, 4, 0, 1, 2, 0, 1, 2, 0, 1, 2, 3, 0, 1, 0, 1, 2,
       3, 0, 1, 2, 3, 0, 1, 2, 3, 4, 0, 1, 2, 0, 1, 2, 3, 0, 1, 2, 3, 4,
       5, 0, 1, 2, 3, 0, 1, 2, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 0, 1, 2, 3,
       0, 1, 2, 3, 0, 1, 0, 1, 2, 0, 1, 2, 3, 4, 5, 0, 1, 2, 0, 1, 2, 0,
       1, 2, 3, 4, 5, 0, 1, 2, 0, 1, 2, 3, 0, 1, 0,

In [17]:
# val_graphs
# basename="163_A1a"
# graph_ids==basename

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [19]:
# np.where(graph_ids==basename)[0]

array([80, 81])

In [6]:
# for now just consider marginalized posterior, in future, add random sampling from posterior
# loss for now will be MSE to logits???
import glob, os
import pandas as pd

# too slow, need to execute this all in parallel in separate nodes and combine results

graph_ids=np.array(list(map(lambda x: x.id,graph_dataset['graph_dataset'])))#[val_idx]
components=np.zeros(len(graph_ids))#[np.arange(n_times)]
# n_times=dict(pd.Series(graph_ids).value_counts())
for ID in np.unique(graph_ids):
    slide_bool=graph_ids==ID
    n_times=sum(slide_bool)
    components[slide_bool]=np.arange(n_times)
components=components.astype(int)
components=components[np.hstack((val_idx,test_idx))]
graph_ids=graph_ids[np.hstack((val_idx,test_idx))]

threshold=225
n_components=2
base_nm="163_A1*"
files=sorted(glob.glob(f"bcc/inputs/{base_nm}.npy"),key=lambda x: x[-5])
for f in files:
    basename=os.path.basename(f).replace(".npy","")#"140_A1c"
    print("Loading Slide")
    arr=np.load(f"bcc/inputs/{basename}.npy")
    arr[cv2.cvtColor(arr,cv2.COLOR_RGB2GRAY)>threshold,:]=255
    pred_map=np.zeros(arr.shape[:2])
    division_map=np.zeros(arr.shape[:2])
    
    slide_idx=np.where(graph_ids==basename)[0]
    n_components=len(slide_idx)
    
    for k in slide_idx:# components[graph_ids==basename]:#range(n_components):
        j=components[k]
        print(f"Processing Slide [{j+1}/{n_components}]")
    #     j=0
#         idx=val_idx[j]

        graph=all_graphs[k]

        print("Getting new interpolation points")

        xmin,ymin=graph['xy'].min(0).astype(int)#graph_dataset['graph_dataset'][idx].pos.values.numpy()
        xmax,ymax=graph['xy'].max(0).astype(int)

        grid=np.array(list(product(range(xmin,xmax,32),range(ymin,ymax,32))))

        G=radius_neighbors_graph(np.vstack([graph['xy'],grid]),radius=16) # then deduplicate and add .. hmm maybe just build off of current graph
        grid=np.delete(grid,G.nonzero()[1][:int(len(G.nonzero()[0])/2)]-len(graph['xy']),axis=0)

        G=radius_neighbors_graph(np.vstack([graph['xy'],grid]),radius=256*np.sqrt(2)) # then deduplicate and add .. hmm maybe just build off of current graph

        new_pos=np.vstack([graph['xy'],grid])[np.where(np.array(G[:,:len(graph['xy'])].sum(1)>0).flatten())[0],:]

        G=radius_neighbors_graph(new_pos,radius=32*np.sqrt(2))

        edge_index=from_scipy_sparse_matrix(G)[0]

        x_old=graph['y_pred']
        knn=KNeighborsClassifier(n_neighbors=1).fit(graph['xy'],x_old.argmax(1))
        prior=knn.predict(new_pos[len(graph['xy']):])
        x_new=np.zeros((G.shape[0]-len(graph['xy']),x_old.shape[1]))
        x_new[:,1]=prior
        x_new[:,0]=1-prior
        # 
        x=torch.tensor(np.vstack([x_old,x_new])).float()
        mask=np.array([False]*len(x))
        mask[:len(graph['xy'])]=True

        print("Propagating Probability Map")

        graph_data=Data(x=x,edge_index=edge_index,mask=torch.tensor(mask),pos=torch.tensor(new_pos))

        graph_data.x=graph_data.x.cuda()
        graph_data.mask=graph_data.mask.cuda()
        graph_data.edge_index=graph_data.edge_index.cuda()

        net=Net(n_layers=30,
               K=15,
               alpha=0.1,
               self_loops=False).cuda()

        y_pred=F.softmax(net(graph_data.x,graph_data.edge_index,graph_data.mask))

#         idx=val_idx[j]
        # graph_dataset['graph_dataset'][idx].id

        preds=y_pred.detach().cpu().numpy()[:,1]

        print("Applying Labels to Map")

        patch_size=256
        for i,(x,y) in tqdm.tqdm(list(enumerate(graph_data.pos.numpy().astype(int).tolist()))):
            pred_map[x:x+patch_size,y:y+patch_size]+=preds[i]
            division_map[x:x+patch_size,y:y+patch_size]+=1.



    print("Blending Final Image")
    division_map[division_map==0]=1
    pred_map=pred_map/division_map
    del division_map
    # arr=cv2.resize(arr,None,fx=1/6.,fy=1/6.)
    small_pred_map=pred_map#cv2.resize(pred_map,None,fx=1/6.,fy=1/6.)
    new_pred_map = cv2.applyColorMap((small_pred_map*255.).astype(np.uint8), cv2.COLORMAP_JET)
    # ADD GAUSSIAN BLUR HERE ^^^ in future
    alpha=0.3
    blended_image=(arr*(1-alpha)+(255-new_pred_map)*alpha).astype(np.uint8)
    blended_image[small_pred_map==0,:]=arr[small_pred_map==0,:]

    for j,k in enumerate(np.argsort([all_graphs[slide_idx[j]]['xy'].max(0)[1] for j in range(n_components)]).tolist()):#val_graphs[j]
        graph=all_graphs[slide_idx[k]]#val_graphs[k]
        xmin,ymin=graph['xy'].min(0).astype(int)#graph_dataset['graph_dataset'][idx].pos.values.numpy()
        xmax,ymax=graph['xy'].max(0).astype(int)
        cv2.imwrite(f"bcc/interactive_test/{basename}.img.{j}.png",cv2.cvtColor(arr[xmin:xmax,ymin:ymax],cv2.COLOR_BGR2RGB))
        cv2.imwrite(f"bcc/interactive_test/{basename}.prob_map.{j}.png",cv2.cvtColor(blended_image[xmin:xmax,ymin:ymax],cv2.COLOR_BGR2RGB))
    del blended_image, pred_map, new_pred_map, small_pred_map
# next is alignment program & remove white space & combine with other slides alphabetically
# ^ align & ink & white space & order
# then openseadragon, but try this one next

# for j in range(3):

# cut up here, add ordering, which will integrate in final stitching with dynamic alignment

# plt.imshow(blended_image)

# fix right here initialize label with original neighbor

# WHY DO PROBABILITY MAPS CHANGE EVERY ITERATION!!!! HELP!!!! :O FIX FIX FIX See slack

Loading Slide
Processing Slide [1/2]
Getting new interpolation points
Propagating Probability Map
Applying Labels to Map


100%|██████████| 867357/867357 [02:34<00:00, 5623.76it/s]


Processing Slide [2/2]
Getting new interpolation points
Propagating Probability Map
Applying Labels to Map


100%|██████████| 845684/845684 [02:29<00:00, 5669.00it/s]


Blending Final Image
Loading Slide
Processing Slide [1/2]
Getting new interpolation points
Propagating Probability Map
Applying Labels to Map


...

Blending Final Image
Loading Slide
Processing Slide [1/2]
Getting new interpolation points
Propagating Probability Map
Applying Labels to Map


100%|██████████| 840019/840019 [02:29<00:00, 5614.69it/s]


Processing Slide [2/2]
Getting new interpolation points
Propagating Probability Map
Applying Labels to Map


100%|██████████| 831614/831614 [02:27<00:00, 5636.31it/s]


Blending Final Image


In [ ]:
# ! ls bcc/interactive_test
# import subprocess,glob

# for f in glob.glob("bcc/interactive_test/*.png"):
#     subprocess.call("mv {} {}".format(f,os.path.join(os.path.dirname(f),f"{basename}.{os.path.basename(f)}")),shell=True)

In [ ]:
# ! mkdir bcc/interactive_test
# ! pip install git+https://github.com/openzoom/deepzoom.py.git

In [ ]:
! ls -lah bcc/interactive_test/

In [ ]:
val_graphs[1]['xy'].max(0)[1]


In [ ]:
val_graphs[2]['xy'].max(0)


In [ ]:
plt.imshow(cv2.resize(blended_image,None,fx=1/6.,fy=1/6.))

In [ ]:
# decide ordering -> pngx2 per section
# then for each section, tissue mask and point cloud alignment or airlab, borrow tissue mask to crop the probability counterpart

In [ ]:
# alpha=0.3
# blended_image=(arr*(1-alpha)+(255-new_pred_map)*alpha).astype(np.uint8)



In [ ]:
# blended_image[small_pred_map==0,:]=arr[small_pred_map==0,:]
# plt.imshow((arr*(1-alpha)+(255-new_pred_map)*alpha).astype(np.uint8))



In [ ]:
plt.imshow(blended_image)





In [ ]:
# OLD!!!!!!!


In [ ]:
# https://raw.githubusercontent.com/snorkel-team/snorkel/master/snorkel/classification/loss.py

from typing import List, Mapping, Optional

import torch
import torch.nn.functional as F

Outputs = Mapping[str, List[torch.Tensor]]


def cross_entropy_with_probs(
    input: torch.Tensor,
    target: torch.Tensor,
    weight: Optional[torch.Tensor] = None,
    reduction: str = "mean",
) -> torch.Tensor:
    """Calculate cross-entropy loss when targets are probabilities (floats), not ints.

    PyTorch's F.cross_entropy() method requires integer labels; it does accept
    probabilistic labels. We can, however, simulate such functionality with a for loop,
    calculating the loss contributed by each class and accumulating the results.
    Libraries such as keras do not require this workaround, as methods like
    "categorical_crossentropy" accept float labels natively.

    Note that the method signature is intentionally very similar to F.cross_entropy()
    so that it can be used as a drop-in replacement when target labels are changed from
    from a 1D tensor of ints to a 2D tensor of probabilities.

    Parameters
    ----------
    input
        A [num_points, num_classes] tensor of logits
    target
        A [num_points, num_classes] tensor of probabilistic target labels
    weight
        An optional [num_classes] array of weights to multiply the loss by per class
    reduction
        One of "none", "mean", "sum", indicating whether to return one loss per data
        point, the mean loss, or the sum of losses

    Returns
    -------
    torch.Tensor
        The calculated loss

    Raises
    ------
    ValueError
        If an invalid reduction keyword is submitted
    """
    num_points, num_classes = input.shape
    # Note that t.new_zeros, t.new_full put tensor on same device as t
    cum_losses = input.new_zeros(num_points)
    for y in range(num_classes):
        target_temp = input.new_full((num_points,), y, dtype=torch.long)
        y_loss = F.cross_entropy(input, target_temp, reduction="none")
        if weight is not None:
            y_loss = y_loss * weight[y]
        cum_losses += target[:, y].float() * y_loss

    if reduction == "none":
        return cum_losses
    elif reduction == "mean":
        return cum_losses.mean()
    elif reduction == "sum":
        return cum_losses.sum()
    else:
        raise ValueError("Keyword 'reduction' must be one of ['none', 'mean', 'sum']")

In [ ]:

    



#y_pred=F.softmax(net(graph_data.x,graph_data.edge_index)).detach().numpy() # ???)
# plt.scatter(*graph_data.pos.numpy().T.tolist(),c=y_pred.argmax(1),s=1e-5)


In [ ]:
# good enough for now, fix the outside points


plt.scatter(*graph_data.pos.numpy().T.tolist(),c=y_pred.detach().cpu().numpy()[:,1],s=1,alpha=0.2)#.argmax(1)

In [ ]:
plt.scatter(*graph['xy'].T.tolist(),c=graph['y_pred'][:,1],s=1)

In [ ]:
plt.scatter(*graph['xy'].T.tolist(),c=graph['y_pred'].argmax(axis=1),s=1)

In [ ]:
plt.scatter(*graph['xy'].T.tolist(),c=graph['y'],s=1)

In [ ]:
# get all of them from the same slide, then do slicing, then produce mixed image and unmixed image

In [ ]:
plt.imshow(arr)

In [ ]:
plt.imshow(pred_map)

In [ ]:
arr.shape

In [ ]:
plt.imshow(new_pred_map)

In [ ]:
alpha=0.3
plt.imshow((arr*(1-alpha)+(255-new_pred_map)*alpha).astype(np.uint8))

In [ ]:
new_pred_map.min()

In [ ]:
arr.shape

In [ ]:
# remove edges post

In [ ]:
import torch.optim as opt

# optimizer=opt.Adam(net.parameters(),lr=1e-2)
# for i in range(5000):
#     y_true=F.softmax(graph_data.x[graph_data.mask])
#     y_pred=net(graph_data.x,graph_data.edge_index,graph_data.mask)[graph_data.mask]
#     loss=cross_entropy_with_probs(y_pred,y_true)
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     print(loss.item())
#     del y_pred,loss
    

In [ ]:
y_true

In [ ]:
y_pred[len(graph['xy']):]

In [ ]:
y_pred.detach().cpu().numpy()[:,1]

In [ ]:
# use a naive untrained GNN adding layers for label propagation?

In [ ]:
#for i,j in G.nonzero()

In [ ]:
graph_dataset['graph_dataset'][0]